---
title: "Security Incidents in Afghanistan: Trends and Patterns"
format:
  html:
    toc: true
    toc-depth: 3
    code-fold: true
    number-sections: true
    theme: cosmo
    fig-width: 9
    fig-height: 6
    fig-dpi: 300
    self-contained: true
bibliography: references.bib
execute:
  warning: false
  message: false
---


<style>
.typing-line {
  font-family: monospace;
  overflow: hidden;
  white-space: nowrap;
  border-right: .15em solid #000;
  width: 0;
  display: block;
  animation: typing 2s steps(20, end) forwards;
  /* Only type, no infinite blinking */
}

.typing-line:nth-child(2) {
  animation-delay: 2s;
}

.typing-line:nth-child(3) {
  animation-delay: 4s;
}

.typing-line:nth-child(4) {
  animation-delay: 6s;
}

@keyframes typing {
  from { width: 0 }
  to { width: 100% }
}
</style>


In [ ]:
#| label: setup
#| include: false
#| echo: false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from matplotlib.colors import LinearSegmentedColormap
import warnings
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
from scipy import stats

# Configure plot settings
plt.rcParams['figure.figsize'] = (10, 6)
warnings.filterwarnings('ignore', category=FutureWarning)
sns.set_theme(style="whitegrid")

# Set consistent color palette
palette = {"primary": "#FFBD59", "secondary": "#64C1FF", "danger": "#FF5252"}

# Load and prepare data
df = pd.read_csv("data/security_incidents_cleaned.csv")
country_data = df[df['country'] == 'Afghanistan']

![](images/afghanpic.png)


In [ ]:
#| echo: false

from IPython.display import HTML

killed = int(country_data['nationals_killed'].sum() + country_data['internationals_killed'].sum())
wounded = int(country_data['nationals_wounded'].sum() + country_data['internationals_wounded'].sum())
kidnapped = int(country_data['nationals_kidnapped'].sum() + country_data['internationals_kidnapped'].sum())

from IPython.display import HTML

html = f"""
<div>
  <span class="typing-line">In Afghanistan...</span>
  <span class="typing-line">{killed} humanitarian workers have died,</span>
  <span class="typing-line">{wounded} humanitarian workers have been wounded,</span>
  <span class="typing-line">{kidnapped} humanitarian workers have been kidnapped.</span>
</div>
"""

HTML(html)

# Executive Summary

![](images/afflag.png){width=200px}

# Background: The Conflict

## Historical Context

Afghanistan has experienced over four decades of continuous conflict since the Soviet invasion in 1979. Following the Soviet withdrawal in 1989, the country descended into civil war, eventually leading to Taliban rule from 1996 to 2001. After the September 11, 2001 attacks, a US-led coalition overthrew the Taliban regime, establishing the Islamic Republic of Afghanistan. The subsequent two decades saw ongoing insurgency, with the Taliban gradually regaining control of territory despite the presence of international forces [@cfr_afghanistan]. Key developments included:

-   **2001-2014**: NATO-led International Security Assistance Force (ISAF) mission
-   **2014-2021**: NATO Resolute Support Mission focused on training Afghan forces
-   **2020**: US-Taliban peace agreement signed in Doha, Qatar
-   **August 2021**: Taliban takeover following US and NATO withdrawal

## Recent Escalation

Following the Taliban's return to power in August 2021, Afghanistan has faced multiple interconnected crises:

-   Approximately 28 million people requiring humanitarian assistance
-   Political transformation under Taliban governance
-   Economic collapse and banking sector crisis
-   Severe humanitarian emergency affecting most of the population
-   Increasing security threats from ISIS-Khorasan (ISIS-K)
-   Natural disasters including earthquakes and drought
-   International isolation and sanctions affecting aid delivery

These factors have created a challenging operational environment for humanitarian organizations, with significant implications for security risk management.

<h1>Map of Incidents <img src="images/map.png" width="60px" style="vertical-align:middle; margin-left:10px;"></h1>


In [ ]:
#| echo: false

icon_path = "images/hazard2.png"
valid_points = country_data[country_data['latitude'].notna() & country_data['longitude'].notna()]

country_map = folium.Map(location=[31.9, 35.2], zoom_start=8)

coords = []

for _, row in valid_points.iterrows():
    coords.append((row['latitude'], row['longitude']))
    
    popup_html = f"""
    <b>Location:</b> {row['location']}, {row['country']}<br>
    <b>Year:</b> {row['year']}<br>
    <b>Total Affected:</b> {row['total_affected']}<br>
    <b>Attack Type:</b> {row.get('means_of_attack', 'Unknown')}
    """

    icon = folium.CustomIcon(
        icon_image=icon_path,
        icon_size=(20, 20)
    )

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=icon
    ).add_to(country_map)

if coords:
    country_map.fit_bounds(coords)

country_map

# Temporal Trends in Security Incidents {#sec-temporal}

## Incident Frequency Over Time


In [ ]:
#| label: fig-yearly-incidents
#| fig-cap: Security incidents
#| echo: false

# Group incidents by year
yearly_incidents = country_data.groupby('year').size().reset_index(name='incidents')

# Create line chart
fig = px.line(
    yearly_incidents, 
    x='year', 
    y='incidents',
    markers=True,
    labels={'incidents': 'Number of Incidents', 'year': 'Year'},
    height=500
)

# Configure layout
fig.update_layout(
    title=None,
    xaxis=dict(tickmode='linear', dtick=2),
    yaxis=dict(zeroline=True),
    hovermode='x unified',
    template='plotly_white'
)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)

# Add Afghanistan-specific event annotations
events = [
    {'year': 2001, 'event': 'US invasion after 9/11', 'y_position': 5},
    {'year': 2009, 'event': 'US troop surge', 'y_position': 27},
    {'year': 2014, 'event': 'NATO combat mission ends', 'y_position': 57},
    {'year': 2018, 'event': 'First peace talks with Taliban', 'y_position': 20},
    {'year': 2021, 'event': 'US withdrawal, Taliban return', 'y_position': 29}
]

for event in events:
    fig.add_annotation(
        x=event['year'],
        y=event['y_position'],
        text=event['event'],
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-20,
        font=dict(size=10),
        bgcolor="rgba(255,255,255,0.85)",
        bordercolor="black",
        borderwidth=1,
        borderpad=3
    )

fig.show()

### Analysis of Incident Trends

The line chart reveals a distinctive pattern in security incidents in Afghanistan:

-   **Initial low levels (1996-2002)**: Following Taliban rule and early in the US invasion, security incidents affecting humanitarian workers remained relatively low.

-   **Steady increase (2003-2009)**: A gradual rise in incidents coincided with expanding insurgency operations and the US troop surge, reflecting deteriorating security conditions.

-   **Peak period (2010-2014)**: Incident levels reached their highest point during the later years of NATO combat operations, with a dramatic spike in 2013 representing the most dangerous period for humanitarian operations.

-   **Sharp decline (2015-2016)**: Following the transition to the NATO Resolute Support Mission and reduced international military footprint, incidents declined significantly.

-   **Fluctuating lower levels (2017-2025)**: Despite periodic increases, incidents have generally remained lower than during the peak conflict years, with a continued downward trend following the Taliban's return to power in 2021.

This pattern suggests that humanitarian security incidents were paradoxically more frequent during periods of intensive international military operations than during Taliban governance, challenging conventional assumptions about operational security risk factors in Afghanistan.

# Nature of Security Incidents {#sec-nature}

## Predominant Attack Types


In [ ]:
#| label: fig-attack-types
#| fig-cap: Most common types of attacks
#| echo: false

attack_counts = country_data['means_of_attack'].value_counts().reset_index()
attack_counts.columns = ['Attack Type', 'Count']
attack_counts = attack_counts.sort_values('Count', ascending=False).head(10)

fig_attacks = px.bar(
    attack_counts,
    y='Attack Type',
    x='Count',
    title=None,
    height=500,
    orientation='h'
)

fig_attacks.update_traces(marker_color=palette["secondary"])

fig_attacks.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title="Number of Incidents",
    yaxis_title=None
)

fig_attacks.show()

### Analysis of Attack Methods

- **Kidnapping dominance**: The high frequency of kidnapping incidents reflects Afghanistan's complex security environment where abductions serve multiple purposes - from financial gain through ransoms to political leverage. This creates significant risks for both international and local personnel.

- **Armed violence**: Shooting incidents represent the second most frequent attack type, indicating the widespread availability of firearms and their regular use in security incidents. This aligns with Afghanistan's history of armed conflict and the proliferation of weapons.

## Attack Contexts


In [ ]:
#| label: fig-attack-contexts
#| fig-cap: Security incidents by attack context
#| echo: false

context_counts = country_data['attack_context'].value_counts().reset_index()
context_counts.columns = ['Attack Context', 'Count']

context_counts = context_counts.sort_values('Count', ascending=False)

fig_context = px.bar(
    context_counts,
    x='Count',
    y='Attack Context',
    title=None,
    height=450,
    orientation='h'
)

fig_context.update_traces(marker_color=palette["secondary"])

fig_context.update_layout(
    xaxis_title="Number of Incidents",
    yaxis_title=None,
    yaxis=dict(categoryorder='total ascending'),  # <-- ensures most at top
    margin=dict(l=150, r=20, t=30, b=30)
)

fig_context.show()

### Analysis of Attack Contexts

- **Ambushes** represent a significant proportion of incidents, indicating the ongoing active conflict environment
- **Direct targeted attacks and raids** account for many incidents, showing deliberate targeting rather than collateral impact

## Location of Attack


In [ ]:
#| label: fig-attack-locations
#| fig-cap: Most common attack locations
#| echo: false

# Get top 10 most frequent locations
location_counts = country_data['location'].value_counts().reset_index()
location_counts.columns = ['Location', 'Count']
location_counts = location_counts.sort_values('Count', ascending=False).head(10)

# Create bar chart
fig_locations = px.bar(
    location_counts,
    y='Location',
    x='Count',
    title=None,
    height=500,
    orientation='h'
)

fig_locations.update_traces(marker_color="#64C1FF")  # <- updated color

fig_locations.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title="Number of Incidents",
    yaxis_title=None
)

fig_locations.show()

### Analysis of Attack Locations

- **Roads** are the most common setting for incidents, underscoring ongoing risks associated with travel.
- A significant number of incidents are marked as unknown, revealing critical gaps in situational reporting and data collection.

# Human Impact of Security Incidents {#sec-impact}

## Casualties by Category


In [ ]:
#| label: fig-casualties
#| fig-cap: Total casualties by category
#| echo: false

stacked_data = {
    'Category': ['Nationals', 'Internationals'],
    'Killed': [
        country_data['nationals_killed'].sum(),
        country_data['internationals_killed'].sum()
    ],
    'Wounded': [
        country_data['nationals_wounded'].sum(),
        country_data['internationals_wounded'].sum()
    ],
    'Kidnapped': [
        country_data['nationals_kidnapped'].sum(),
        country_data['internationals_kidnapped'].sum()
    ]
}

stacked_df = pd.DataFrame(stacked_data)
stacked_df_long = pd.melt(
    stacked_df,
    id_vars=['Category'],
    value_vars=['Killed', 'Wounded', 'Kidnapped'],
    var_name='Status',
    value_name='Count'
)

fig_stacked = px.bar(
    stacked_df_long,
    x='Category',
    y='Count',
    color='Status',
    title=None,
    barmode='stack',
    height=500,
    color_discrete_map={
        'Killed': palette["danger"], 
        'Wounded': palette["primary"], 
        'Kidnapped': palette["secondary"]
    }
)

fig_stacked.update_layout(
    xaxis_title=None,
    yaxis_title="Number of People",
    legend_title_text=None
)

fig_stacked.show()

### Analysis of Casualty Patterns

The stacked bar chart reveals a stark disparity in how security incidents affect different population groups in Afghanistan:

- Afghan nationals have suffered significantly higher casualties across all categories

This pattern is not unusual in conflict zones, where local populations typically bear the brunt of violence. However, the scale of the difference is notable.

# Major Incidents {#sec-major}

## Top 5 Most Impactful Security Incidents


In [ ]:
#| label: tbl-interactive-incidents
#| echo: false

import plotly.graph_objects as go
from IPython.display import display, HTML

if not country_data.empty and 'total_affected' in country_data.columns:
    # Sort the entire dataset by total_affected
    sorted_incidents = country_data.sort_values(by='total_affected', ascending=False)
    
    # Select display columns and rename them
    display_cols = ['year', 'location', 'total_affected', 'total_killed', 'total_wounded', 'attack_context', 'actor_type']
    column_names = {
        'year': 'Year',
        'location': 'Location',
        'total_affected': 'Total Affected',
        'total_killed': 'Killed',
        'total_wounded': 'Wounded',
        'attack_context': 'Context',
        'actor_type': 'Actor Type'
    }
    
    # Prepare the display dataframe
    display_df = sorted_incidents[display_cols].rename(columns=column_names)
    
    # Create an interactive table with Plotly
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=list(column_names.values()),
            fill_color='#A1083B',
            align='left',
            font=dict(color='white', size=14),
            height=40
        ),
        cells=dict(
            values=[display_df[col] for col in display_df.columns],
            fill_color=[['#F99301' if i < 5 else '#f9f9f9' for i in range(len(display_df))]],
            align='left',
            font=dict(color=['black' if i < 5 else '#333333' for i in range(len(display_df))], size=12),
            height=30
        )
    )])
    
    # Update layout for better appearance with fixed height for scrolling
    fig.update_layout(
        title={
            'text': 'Security Incidents Ranked by Impact',
            'y': 0.98,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20}
        },
        margin=dict(l=20, r=20, t=60, b=20),
        height=600,  # Fixed height to enable scrolling
        width=900
    )
    
    # Add a note about the highlighted rows
    fig.add_annotation(
        x=0,
        y=-0.15,
        xref='paper',
        yref='paper',
        text="<i>Top 5 most impactful incidents are highlighted. Scroll to view all incidents.</i>",
        showarrow=False,
        font=dict(size=12),
        align='left'
    )
    
    # Save and display the figure
    fig.write_html("images/interactive_incidents_table.html",
                  include_plotlyjs='cdn',
                  config={'scrollZoom': True})
    fig.show()
    
    # Removed the display(HTML(search_html)) line and the search_html variable

### Analysis of Major Incidents

The table above highlights the five most impactful security incidents in Afghanistan based on the total number of people affected. These high-casualty events reveal several important patterns:

-   The most severe incident occurred in 2015 at a project site, resulting in 49 casualties (14 killed and 35 wounded), indicating the vulnerability of development projects in Afghanistan
-   The incidents span from 2009 to 2021, demonstrating that high-impact security events have been a persistent feature of the Afghan security landscape across different phases of the conflict
-   Three of the five top incidents resulted in fatalities, while two incidents (both with 12 affected individuals) show no deaths or injuries, suggesting these were likely kidnapping incidents

These high-impact events, while relatively rare, account for a disproportionate share of total casualties.

For more detailed information on the Afghanistan conflict timeline, see the [Council on Foreign Relations Conflict Tracker](https://www.cfr.org/global-conflict-tracker/conflict/war-afghanistan).

# Conclusions and Recommendations {#sec-conclusions}

## Key Findings

This analysis of security incidents in Afghanistan reveals several critical patterns and trends:

1.  **Changing patterns over time**: Security incidents peaked during intensive NATO operations (2010-2014) and have generally declined since, with further reduction following Taliban control in 2021.

2.  **Targeted attacks predominate**: Unlike Palestine's collateral damage pattern, Afghanistan shows more deliberate targeting of humanitarian operations, particularly compounds and convoys.

3.  **High-impact events**: Major incidents at fixed locations account for a disproportionate share of casualties, highlighting the vulnerability of permanent facilities.

4.  **Kidnapping dynamics**: Afghanistan shows a distinctive pattern of kidnapping incidents, particularly along roads and transport routes.

## 💡 Recommendations 

These findings suggest specific approaches for operating in the Afghan context:

### For Security Planning

-   Implement robust physical security measures for compounds and offices
-   Develop comprehensive convoy security protocols with route risk assessments
-   Maintain low-profile operations in high-risk areas
-   Establish clear kidnapping prevention and response procedures

### For Humanitarian Organizations

-   Balance operational effectiveness with security considerations when determining fixed vs. mobile programming
-   Invest in local acceptance strategies, especially given the changing political landscape
-   Develop contingency plans for periods of heightened risk

### For Future Research

-   Examine the relationship between political transitions and security incident patterns
-   Investigate the effectiveness of different security approaches in the post-2021 context
-   Compare security incidents affecting humanitarian versus development actors

By providing a data-driven understanding of security patterns in Afghanistan, this analysis aims to contribute to safer and more effective operations in this challenging environment.